### 任务1：构造一个2层的全连接神经网络，用于Mnist手写数字识别
+ Mnist手写数据集包括60,000个训练样本和10,000个测试样本组成，每个样本是一张28*28像素的灰度手写数字图片

#### 导入所需的python包

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")
print("Visible device", tf.config.experimental.get_visible_devices())

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

GPU is available
Visible device [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.0.0


#### 第一步：加载并预处理数据集——用tf.function构建AutoGraph

In [2]:
@tf.function
def preprocess_mninst(x,y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x,y

@tf.function
def mnist_dataset():
    (train_images, train_labels),(test_images, test_labels) = keras.datasets.mnist.load_data()
    ds_train = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    ds_train = ds_train.map(preprocess_mninst)
    ds_train = ds_train.take(60000).shuffle(60000).batch(128)
    
    ds_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    ds_test = ds_test.map(preprocess_mninst)
    ds_test = ds_test.take(10000).shuffle(10000).batch(128)
    return ds_train, ds_test
    

#### 第二步：使用tensorflow2.0构造一个全连接神经网络

In [3]:
# 继承Model类后，使用keras.layers构造模型计算图
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = layers.Conv2D(32, 3, activation='relu')
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(128, activation='relu')
        self.d2 = layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

model = MyModel()

# 选择优化算法
optimizer = keras.optimizers.Adam()

# 选择损失函数
compute_loss = keras.losses.SparseCategoricalCrossentropy()

# 选择度量模型好坏的指标，损失loss和准确率accuracy。这些指标在 epoch 上累积值，然后打印出整体结果。
train_loss = keras.metrics.Mean(name="train_loss")
train_accuracy = keras.metics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# # 选择损失函数
# @tf.function
# def compute_loss(predict, label):
#     return tf.reduce_mean(keras.losses.sparse_categorical_crossentropy(label, predict))

# # 选择度量模型好坏的指标，损失loss和准确率accuracy
# @tf.function
# def compute_metrics(predict, label):
#     pred_value = tf.argmax(predict, axis=1)
#     accuracy = tf.reduce_mean(tf.cast(tf.equal(pred_value, label), tf.float32))
#     return accuracy


AttributeError: module 'tensorflow_core.keras' has no attribute 'metics'

#### 第三步：构造训练函数

In [ ]:
# 单次训练函数
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)                                         # 前向传播过程
        loss = compute_loss(labels, predictions)                       # 计算损失函数
    grads = tape.gradient(loss, model.trainable_variables)             # 反向传播计算梯度
    optimizer.apply_gradients(zip(grads, model.trainable_variables))   # 更新权重参数
    
    train_loss(loss)                                                   # 计算训练集上的平均损失
    train_accuracy(labels, predictions)                                # 计算训练集上的准确率

    
# 单次测试函数
@tf.function
def test_step(x, y):
    predictions = model(x)
    loss = compute_loss(y, predictions)
    
    test_loss(loss)
    test_accuracy(y, predictions)

# 开始训练：
EPOCHS = 10
for epoch in range(EPOCHS):
    ds_train, ds_test = mnist_dataset()
    for step, (x,y) in enumerate(ds_train):
        train_step(x, y)
    for step, (x,y) in enumerate(ds_test):
        test_step(x, y) 
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))    
    